In [ ]:
"""
Trello YouTube Downloader & Uploader

A Python library for downloading YouTube videos from URLs found in Trello cards
and uploading them back to the cards as attachments.
"""

import requests
import re
import os
import json
import yt_dlp
from typing import Dict, List, Optional, Tuple
from dotenv import load_dotenv
from urllib.parse import urlparse, parse_qs
import logging
import glob
from pathlib import Path


class TrelloYouTubeDownloader:
    """
    A class to download YouTube videos from URLs found in Trello cards
    and upload them back to the cards as attachments.
    
    This class provides functionality to:
    - Fetch all cards from a Trello list
    - Extract YouTube URLs from card titles and descriptions
    - Download videos using yt-dlp
    - Upload downloaded videos as attachments to the source cards
    - Track download and upload progress and errors
    """
    
    def __init__(self, download_path: str = "./downloads", keep_local_files: bool = True, 
                 destination_list_id: Optional[str] = None):
        """
        Initialize the Trello YouTube Downloader.
        
        Automatically loads credentials from .env file using:
        - TRELLO_API_KEY
        - TRELLO_API_TOKEN
        
        Args:
            download_path (str): Directory to save downloaded videos
            keep_local_files (bool): Whether to keep local files after upload
            destination_list_id (Optional[str]): List ID to move successfully processed cards to
        """
        # Load environment variables from .env file
        load_dotenv()
        
        self.api_key = os.getenv("TRELLO_API_KEY")
        self.token = os.getenv("TRELLO_API_TOKEN")
        
        if not self.api_key:
            raise ValueError("TRELLO_API_KEY not found in environment variables")
        if not self.token:
            raise ValueError("TRELLO_API_TOKEN not found in environment variables")
            
        self.base_url = "https://api.trello.com/1"
        self.download_path = download_path
        self.keep_local_files = keep_local_files
        self.destination_list_id = destination_list_id
        
        # Create download directory if it doesn't exist
        os.makedirs(self.download_path, exist_ok=True)
        
        # Setup logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        
        # YouTube URL patterns
        self.youtube_patterns = [
            r'(?:https?://)?(?:www\.)?youtube\.com/watch\?v=([a-zA-Z0-9_-]+)',
            r'(?:https?://)?(?:www\.)?youtu\.be/([a-zA-Z0-9_-]+)',
            r'(?:https?://)?(?:www\.)?youtube\.com/embed/([a-zA-Z0-9_-]+)',
            r'(?:https?://)?(?:www\.)?youtube\.com/v/([a-zA-Z0-9_-]+)',
            r'(?:https?://)?(?:www\.)?youtube\.com/shorts/([a-zA-Z0-9_-]+)'
        ]
    
    def get_cards_from_list(self, list_id: str) -> List[Dict]:
        """
        Fetch all cards from the specified Trello list.
        
        Args:
            list_id (str): The ID of the Trello list
            
        Returns:
            List[Dict]: List of card data dictionaries
            
        Raises:
            requests.RequestException: If the API request fails
        """
        url = f"{self.base_url}/lists/{list_id}/cards"
        params = {
            'key': self.api_key,
            'token': self.token,
            'fields': 'id,name,desc,shortUrl'
        }
        
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            cards = response.json()
            self.logger.info(f"Fetched {len(cards)} cards from list {list_id}")
            return cards
        except requests.RequestException as e:
            self.logger.error(f"Error fetching cards: {e}")
            raise
    
    def extract_youtube_urls(self, text: str) -> List[str]:
        """
        Extract YouTube URLs from text using regex patterns.
        
        Args:
            text (str): Text to search for YouTube URLs
            
        Returns:
            List[str]: List of found YouTube URLs (normalized)
        """
        urls = []
        if not text:
            return urls
            
        for pattern in self.youtube_patterns:
            matches = re.finditer(pattern, text, re.IGNORECASE)
            for match in matches:
                video_id = match.group(1)
                # Normalize to standard YouTube URL format
                normalized_url = f"https://www.youtube.com/watch?v={video_id}"
                if normalized_url not in urls:
                    urls.append(normalized_url)
        
        return urls
    
    def get_youtube_urls_from_card(self, card: Dict) -> List[str]:
        """
        Extract all YouTube URLs from a card's title and description.
        
        Args:
            card (Dict): Card data dictionary
            
        Returns:
            List[str]: List of YouTube URLs found in the card
        """
        urls = []
        
        # Check card title
        title_urls = self.extract_youtube_urls(card.get('name', ''))
        urls.extend(title_urls)
        
        # Check card description
        desc_urls = self.extract_youtube_urls(card.get('desc', ''))
        urls.extend(desc_urls)
        
        # Remove duplicates while preserving order
        unique_urls = []
        for url in urls:
            if url not in unique_urls:
                unique_urls.append(url)
                
        return unique_urls
    
    def sanitize_filename(self, filename: str) -> str:
        """
        Sanitize filename by removing/replacing invalid characters.
        
        Args:
            filename (str): Original filename
            
        Returns:
            str: Sanitized filename safe for filesystem
        """
        # Remove or replace invalid characters
        invalid_chars = '<>:"/\\|?*'
        for char in invalid_chars:
            filename = filename.replace(char, '_')
        
        # Limit length and strip whitespace
        filename = filename.strip()[:200]
        
        return filename
    
    def upload_file_to_card(self, card_id: str, file_path: str, file_name: str = None) -> Dict:
        """
        Upload a file as an attachment to a Trello card.
        
        Args:
            card_id (str): The ID of the Trello card
            file_path (str): Path to the file to upload
            file_name (str): Optional custom name for the attachment
            
        Returns:
            Dict: Upload result with status and details
        """
        if not os.path.exists(file_path):
            return {
                'status': 'error',
                'error': f'File not found: {file_path}',
                'file_path': file_path
            }
        
        url = f"{self.base_url}/cards/{card_id}/attachments"
        
        # Get file size for validation
        file_size = os.path.getsize(file_path)
        max_size = 250 * 1024 * 1024  # 250MB limit for Trello
        
        if file_size > max_size:
            return {
                'status': 'error',
                'error': f'File too large: {file_size} bytes (max: {max_size} bytes)',
                'file_path': file_path,
                'file_size': file_size
            }
        
        try:
            # Use the original filename if no custom name provided
            if not file_name:
                file_name = os.path.basename(file_path)
            
            with open(file_path, 'rb') as file:
                files = {
                    'file': (file_name, file, 'application/octet-stream')
                }
                data = {
                    'key': self.api_key,
                    'token': self.token,
                    'name': file_name
                }
                
                self.logger.info(f"Uploading {file_name} ({file_size} bytes) to card {card_id}")
                
                response = requests.post(url, files=files, data=data)
                response.raise_for_status()
                
                attachment_info = response.json()
                
                return {
                    'status': 'success',
                    'attachment_id': attachment_info['id'],
                    'attachment_url': attachment_info['url'],
                    'file_name': file_name,
                    'file_path': file_path,
                    'file_size': file_size
                }
                
        except requests.RequestException as e:
            return {
                'status': 'error',
                'error': f'Upload failed: {str(e)}',
                'file_path': file_path
            }
        except Exception as e:
            return {
                'status': 'error',
                'error': f'Unexpected error: {str(e)}',
                'file_path': file_path
            }
    
    def move_card_to_list(self, card_id: str, destination_list_id: str) -> Dict:
        """
        Move a card to another list.
        
        Args:
            card_id (str): The ID of the card to move
            destination_list_id (str): The ID of the destination list
            
        Returns:
            Dict: Move result with status and details
        """
        url = f"{self.base_url}/cards/{card_id}"
        params = {
            'key': self.api_key,
            'token': self.token
        }
        data = {
            'idList': destination_list_id
        }
        
        try:
            response = requests.put(url, params=params, data=data)
            response.raise_for_status()
            
            self.logger.info(f"Successfully moved card {card_id} to list {destination_list_id}")
            
            return {
                'status': 'success',
                'card_id': card_id,
                'destination_list_id': destination_list_id
            }
            
        except requests.RequestException as e:
            self.logger.error(f"Failed to move card {card_id}: {str(e)}")
            return {
                'status': 'error',
                'card_id': card_id,
                'destination_list_id': destination_list_id,
                'error': str(e)
            }
    
    def download_video(self, url: str, card_info: Dict) -> Dict:
        """
        Download a single YouTube video.
        
        Args:
            url (str): YouTube URL to download
            card_info (Dict): Information about the source card
            
        Returns:
            Dict: Download result with status, details, and file paths
        """
        try:
            # Create subdirectory for this card
            card_name = self.sanitize_filename(card_info['name'])
            card_dir = os.path.join(self.download_path, f"card_{card_info['id']}_{card_name}")
            os.makedirs(card_dir, exist_ok=True)
            
            # yt-dlp options
            ydl_opts = {
                'outtmpl': os.path.join(card_dir, '%(title)s.%(ext)s'),
                'format': 'best[height<=1080][filesize<250M]/best[height<=720]/best',  # Allow up to 1080p within 250MB limit
                'writesubtitles': True,  # Re-enable subtitles since we have more space
                'writeautomaticsub': True,
                'subtitleslangs': ['en'],
                'ignoreerrors': True,
                'no_warnings': False,
            }
            
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                # Get video info first
                info = ydl.extract_info(url, download=False)
                video_title = info.get('title', 'Unknown Title')
                duration = info.get('duration', 0)
                
                self.logger.info(f"Downloading: {video_title} ({duration}s) from card: {card_info['name']}")
                
                # Download the video
                ydl.download([url])
                
                # Find downloaded files
                downloaded_files = []
                for ext in ['mp4', 'webm', 'mkv', 'avi', 'mov']:
                    pattern = os.path.join(card_dir, f"*.{ext}")
                    files = glob.glob(pattern)
                    downloaded_files.extend(files)
                
                return {
                    'status': 'success',
                    'url': url,
                    'title': video_title,
                    'duration': duration,
                    'download_path': card_dir,
                    'downloaded_files': downloaded_files,
                    'card_info': card_info
                }
                
        except Exception as e:
            self.logger.error(f"Error downloading {url}: {str(e)}")
            return {
                'status': 'error',
                'url': url,
                'error': str(e),
                'card_info': card_info,
                'downloaded_files': []
            }
    
    def process_card_videos(self, card: Dict, max_videos_per_card: int = 5) -> Dict:
        """
        Process all YouTube videos for a single card: download and upload.
        
        Args:
            card (Dict): Card information
            max_videos_per_card (int): Maximum videos to process per card
            
        Returns:
            Dict: Processing results for this card
        """
        card_results = {
            'card_info': card,
            'youtube_urls': [],
            'downloads': [],
            'uploads': [],
            'move_result': None,
            'processing_successful': False,
            'errors': []
        }
        
        try:
            # Extract YouTube URLs from card
            youtube_urls = self.get_youtube_urls_from_card(card)
            card_results['youtube_urls'] = youtube_urls
            
            if not youtube_urls:
                return card_results
            
            self.logger.info(f"Found {len(youtube_urls)} YouTube URLs in card: {card['name']}")
            
            # Limit number of videos per card
            urls_to_process = youtube_urls[:max_videos_per_card]
            if len(youtube_urls) > max_videos_per_card:
                self.logger.warning(f"Card has {len(youtube_urls)} videos, limiting to {max_videos_per_card}")
            
            # Process each video: download then upload
            for url in urls_to_process:
                # Download video
                download_result = self.download_video(url, card)
                card_results['downloads'].append(download_result)
                
                if download_result['status'] == 'success':
                    # Upload each downloaded file to the card
                    for file_path in download_result['downloaded_files']:
                        upload_result = self.upload_file_to_card(
                            card['id'], 
                            file_path,
                            f"Video: {os.path.basename(file_path)}"
                        )
                        upload_result['source_url'] = url
                        upload_result['video_title'] = download_result['title']
                        card_results['uploads'].append(upload_result)
                        
                        if upload_result['status'] == 'success':
                            self.logger.info(f"Successfully uploaded {os.path.basename(file_path)} to card {card['name']}")
                            
                            # Delete local file if not keeping them
                            if not self.keep_local_files:
                                try:
                                    os.remove(file_path)
                                    self.logger.info(f"Deleted local file: {file_path}")
                                except Exception as e:
                                    self.logger.warning(f"Could not delete {file_path}: {e}")
                        else:
                            self.logger.error(f"Failed to upload {file_path}: {upload_result['error']}")
            
            # Determine if processing was successful
            if card_results['youtube_urls']:
                # Check if we have any successful uploads
                successful_uploads = [u for u in card_results['uploads'] if u['status'] == 'success']
                if successful_uploads:
                    card_results['processing_successful'] = True
                    self.logger.info(f"Card {card['name']} processed successfully with {len(successful_uploads)} video uploads")
                    
                    # Move card to destination list if specified
                    if self.destination_list_id:
                        move_result = self.move_card_to_list(card['id'], self.destination_list_id)
                        card_results['move_result'] = move_result
                        
                        if move_result['status'] == 'success':
                            self.logger.info(f"Card {card['name']} moved to destination list")
                        else:
                            self.logger.error(f"Failed to move card {card['name']}: {move_result['error']}")
                else:
                    self.logger.warning(f"Card {card['name']} had YouTube URLs but no successful uploads")
            else:
                # No YouTube URLs found, but this isn't an error - just log it
                self.logger.info(f"No YouTube URLs found in card {card['name']} - skipping")
                
        except Exception as e:
            error_info = {
                'error': str(e),
                'context': 'card_processing'
            }
            card_results['errors'].append(error_info)
            self.logger.error(f"Error processing card {card['name']}: {str(e)}")
        
        return card_results
    
    def process_list(self, list_id: str, max_videos_per_card: int = 5) -> Dict:
        """
        Master function to process all cards in a list: download YouTube videos, upload to cards, 
        and optionally move successful cards to another list.
        
        Args:
            list_id (str): The ID of the Trello list to process
            max_videos_per_card (int): Maximum number of videos to download per card
            
        Returns:
            Dict: Summary of the processing results
        """
        self.logger.info(f"Starting to process Trello list: {list_id}")
        if self.destination_list_id:
            self.logger.info(f"Will move successful cards to list: {self.destination_list_id}")
        
        results = {
            'list_id': list_id,
            'destination_list_id': self.destination_list_id,
            'total_cards': 0,
            'cards_with_youtube': 0,
            'cards_processed_successfully': 0,
            'cards_moved_successfully': 0,
            'total_videos_found': 0,
            'total_videos_downloaded': 0,
            'total_videos_uploaded': 0,
            'successful_downloads': [],
            'failed_downloads': [],
            'successful_uploads': [],
            'failed_uploads': [],
            'successful_moves': [],
            'failed_moves': [],
            'card_results': [],
            'processing_errors': []
        }
        
        try:
            # Get all cards from the list
            cards = self.get_cards_from_list(list_id)
            results['total_cards'] = len(cards)
            
            for card in cards:
                self.logger.info(f"Processing card: {card['name']} (ID: {card['id']})")
                
                card_results = self.process_card_videos(card, max_videos_per_card)
                results['card_results'].append(card_results)
                
                # Update summary statistics
                if card_results['youtube_urls']:
                    results['cards_with_youtube'] += 1
                    results['total_videos_found'] += len(card_results['youtube_urls'])
                
                # Track successful processing
                if card_results['processing_successful']:
                    results['cards_processed_successfully'] += 1
                
                # Process downloads
                for download in card_results['downloads']:
                    if download['status'] == 'success':
                        results['successful_downloads'].append(download)
                        results['total_videos_downloaded'] += 1
                    else:
                        results['failed_downloads'].append(download)
                
                # Process uploads
                for upload in card_results['uploads']:
                    if upload['status'] == 'success':
                        results['successful_uploads'].append(upload)
                        results['total_videos_uploaded'] += 1
                    else:
                        results['failed_uploads'].append(upload)
                
                # Process move result
                if card_results['move_result']:
                    if card_results['move_result']['status'] == 'success':
                        results['successful_moves'].append(card_results['move_result'])
                        results['cards_moved_successfully'] += 1
                    else:
                        results['failed_moves'].append(card_results['move_result'])
                
                # Process errors
                for error in card_results['errors']:
                    error['card_name'] = card['name']
                    error['card_id'] = card['id']
                    results['processing_errors'].append(error)
            
            # Clean up empty directories if not keeping local files
            if not self.keep_local_files:
                self.cleanup_empty_directories()
            
            # Save results summary
            self.save_results_summary(list_id, results)
            
        except Exception as e:
            self.logger.error(f"Fatal error processing list {list_id}: {str(e)}")
            results['processing_errors'].append({
                'error': f"Fatal error: {str(e)}",
                'context': 'list_processing'
            })
        
        return results
    
    def cleanup_empty_directories(self):
        """Remove empty directories in the download path."""
        try:
            for root, dirs, files in os.walk(self.download_path, topdown=False):
                for directory in dirs:
                    dir_path = os.path.join(root, directory)
                    try:
                        if not os.listdir(dir_path):  # Directory is empty
                            os.rmdir(dir_path)
                            self.logger.info(f"Removed empty directory: {dir_path}")
                    except OSError:
                        pass  # Directory not empty or other error
        except Exception as e:
            self.logger.warning(f"Error during cleanup: {e}")
    
    def save_results_summary(self, list_id: str, results: Dict):
        """
        Save processing results to a JSON file.
        
        Args:
            list_id (str): The processed list ID
            results (Dict): Processing results
        """
        summary_file = os.path.join(self.download_path, f"results_summary_{list_id}.json")
        
        try:
            # Create a serializable copy of results (remove non-serializable objects)
            serializable_results = json.loads(json.dumps(results, default=str))
            
            with open(summary_file, 'w', encoding='utf-8') as f:
                json.dump(serializable_results, f, indent=2, ensure_ascii=False)
            self.logger.info(f"Results summary saved to: {summary_file}")
        except Exception as e:
            self.logger.error(f"Error saving results summary: {str(e)}")
    
    def print_summary(self, results: Dict):
        """
        Print a formatted summary of the processing results.
        
        Args:
            results (Dict): Processing results to summarize
        """
        print("\n" + "="*70)
        print("TRELLO YOUTUBE DOWNLOADER & UPLOADER - SUMMARY")
        print("="*70)
        print(f"List ID: {results['list_id']}")
        if results.get('destination_list_id'):
            print(f"Destination List ID: {results['destination_list_id']}")
        print(f"Total cards processed: {results['total_cards']}")
        print(f"Cards with YouTube videos: {results['cards_with_youtube']}")
        print(f"Cards processed successfully: {results['cards_processed_successfully']}")
        if results.get('destination_list_id'):
            print(f"Cards moved successfully: {results['cards_moved_successfully']}")
        print(f"Total YouTube videos found: {results['total_videos_found']}")
        print(f"Videos successfully downloaded: {results['total_videos_downloaded']}")
        print(f"Videos successfully uploaded to cards: {results['total_videos_uploaded']}")
        print(f"Failed downloads: {len(results['failed_downloads'])}")
        print(f"Failed uploads: {len(results['failed_uploads'])}")
        if results.get('failed_moves'):
            print(f"Failed card moves: {len(results['failed_moves'])}")
        print(f"Processing errors: {len(results['processing_errors'])}")
        
        if results['successful_uploads']:
            print(f"\n✅ SUCCESSFUL UPLOADS ({len(results['successful_uploads'])}):")
            for upload in results['successful_uploads']:
                print(f"  📹 {upload.get('video_title', 'Unknown Title')}")
                print(f"     File: {upload['file_name']}")
                print(f"     Size: {upload.get('file_size', 0)} bytes")
                print(f"     Attachment URL: {upload['attachment_url']}")
                print()
        
        if results['failed_downloads']:
            print(f"\n❌ FAILED DOWNLOADS ({len(results['failed_downloads'])}):")
            for download in results['failed_downloads']:
                print(f"  🔗 {download['url']}")
                print(f"     Card: {download['card_info']['name']}")
                print(f"     Error: {download['error']}")
                print()
        
        if results['failed_uploads']:
            print(f"\n⚠️  FAILED UPLOADS ({len(results['failed_uploads'])}):")
            for upload in results['failed_uploads']:
                print(f"  📁 {upload.get('file_name', 'Unknown file')}")
                print(f"     Error: {upload['error']}")
                print()
        
        if results.get('successful_moves'):
            print(f"\n✅ SUCCESSFUL CARD MOVES ({len(results['successful_moves'])}):")
            for move in results['successful_moves']:
                # Find the card name from card_results
                card_name = "Unknown Card"
                for card_result in results['card_results']:
                    if card_result['card_info']['id'] == move['card_id']:
                        card_name = card_result['card_info']['name']
                        break
                print(f"  📋 {card_name}")
                print(f"     Moved to list: {move['destination_list_id']}")
                print()
        
        if results.get('failed_moves'):
            print(f"\n❌ FAILED CARD MOVES ({len(results['failed_moves'])}):")
            for move in results['failed_moves']:
                # Find the card name from card_results
                card_name = "Unknown Card"
                for card_result in results['card_results']:
                    if card_result['card_info']['id'] == move['card_id']:
                        card_name = card_result['card_info']['name']
                        break
                print(f"  📋 {card_name}")
                print(f"     Target list: {move['destination_list_id']}")
                print(f"     Error: {move['error']}")
                print()
        
        if results['processing_errors']:
            print(f"\n⚠️  PROCESSING ERRORS ({len(results['processing_errors'])}):")
            for error in results['processing_errors']:
                print(f"  📋 {error.get('card_name', 'Unknown card')}")
                print(f"     Error: {error['error']}")
                print()
        
        print("="*70)


def download_and_upload_youtube_videos(list_id: str, download_path: str = "./downloads", 
                                      max_videos_per_card: int = 5, 
                                      keep_local_files: bool = False,
                                      destination_list_id: Optional[str] = None) -> Dict:
    """
    Convenience function to download YouTube videos from a Trello list, upload them to cards,
    and optionally move successful cards to another list.
    
    Args:
        list_id (str): The ID of the Trello list to process
        download_path (str): Directory to save downloaded videos
        max_videos_per_card (int): Maximum number of videos to download per card
        keep_local_files (bool): Whether to keep local files after upload
        destination_list_id (Optional[str]): List ID to move successfully processed cards to
        
    Returns:
        Dict: Processing results summary
    """
    downloader = TrelloYouTubeDownloader(download_path, keep_local_files, destination_list_id)
    results = downloader.process_list(list_id, max_videos_per_card)
    downloader.print_summary(results)
    return results




In [ ]:
list_id = os.getenv("TRELLO_YOUTUBE_LINK_LIST_ID")
destination_list_id = os.getenv("TRELLO_MP4_LINK_LIST_ID")

try:
    results = download_and_upload_youtube_videos(
        list_id=list_id,
        download_path="./youtube_downloads",
        max_videos_per_card=3,
        keep_local_files=False,  # Delete local files after upload
        destination_list_id=destination_list_id  # Move successful cards here
    )
    
    print(f"Processing complete!")
    print(f"Downloaded: {results['total_videos_downloaded']} videos")
    print(f"Uploaded to cards: {results['total_videos_uploaded']} videos")
    print(f"Cards moved: {results['cards_moved_successfully']} cards")
    
except Exception as e:
    print(f"Error: {e}")

# Trello YouTube Downloader & Uploader

## Requirements

Create a `requirements.txt` file:

```txt
requests>=2.28.0
yt-dlp>=2023.7.6
python-dotenv>=1.0.0
```

## Environment Setup

Create a `.env` file in your project directory:

```env
TRELLO_API_KEY=your_trello_api_key_here
TRELLO_API_TOKEN=your_trello_api_token_here
```

## Installation

```bash
pip install -r requirements.txt
```

## Simple Usage Example

```python
from trello_youtube_downloader import download_and_upload_youtube_videos

# Download YouTube videos from a Trello list, upload them as card attachments,
# and move successful cards to another list
list_id = "your_source_list_id_here"
destination_list_id = "your_destination_list_id_here"  # Optional

results = download_and_upload_youtube_videos(
    list_id=list_id,
    destination_list_id=destination_list_id
)
```

## Advanced Usage Example

```python
from trello_youtube_downloader import TrelloYouTubeDownloader

# Create downloader instance with custom settings
downloader = TrelloYouTubeDownloader(
    download_path="./my_videos",
    keep_local_files=True,  # Keep downloaded files after upload
    destination_list_id="your_completed_list_id"  # Move successful cards here
)

# Process a list with custom settings
results = downloader.process_list(
    list_id="your_source_list_id",
    max_videos_per_card=10  # Download up to 10 videos per card
)

# Print detailed summary
downloader.print_summary(results)

# Access detailed results
print(f"Successfully downloaded: {results['total_videos_downloaded']} videos")
print(f"Successfully uploaded: {results['total_videos_uploaded']} videos")
print(f"Cards moved to destination: {results['cards_moved_successfully']} cards")
print(f"From {results['cards_with_youtube']} cards with YouTube content")
```

## How to Get Your Trello API Credentials

1. Go to https://trello.com/app-key
2. Copy your API Key
3. Click the "Token" link to generate a token
4. Copy the token
5. Add both to your `.env` file

## How to Get Trello List IDs

### For Source and Destination Lists:

1. Open your Trello board in a web browser
2. Go to any list you want to get the ID for
3. Click on a card in that list
4. Look at the URL - it will be like: `https://trello.com/c/CARD_ID/card-name`
5. Add `.json` to the end: `https://trello.com/c/CARD_ID/card-name.json`
6. Open that URL in your browser
7. Look for `"idList"` in the JSON - that's your list ID

### Typical Workflow Setup:
- **Source List**: "Videos to Process" (contains cards with YouTube URLs)
- **Destination List**: "Completed" (where successfully processed cards are moved)

This creates a clear workflow where cards move from "To Process" → "Completed" after videos are downloaded and attached.

## Features

- **Complete Workflow**: Downloads YouTube videos AND uploads them as Trello card attachments
- **Workflow Management**: Optionally moves successfully processed cards to another list (perfect for "To Process" → "Completed" workflows)
- **Automatic URL Detection**: Finds YouTube URLs in card titles and descriptions
- **Multiple URL Formats**: Supports youtube.com, youtu.be, shorts, embed URLs
- **Smart File Management**: Organizes downloads and optionally cleans up after upload
- **File Size Optimization**: Downloads videos under 250MB to meet Trello's attachment limits
- **Progress Tracking**: Detailed logging and summary reports for both downloads and uploads
- **Error Handling**: Robust error handling with detailed error reporting
- **Quality Control**: Downloads videos in high quality (up to 1080p) within 250MB limit
- **Batch Processing**: Processes entire lists automatically
- **Summary Reports**: Generates JSON summary files for each processing run
- **Upload Tracking**: Monitors successful and failed uploads to cards

## Download Structure (when keep_local_files=True)

```
downloads/
├── card_123_CardName1/
│   ├── Video Title 1.mp4 (also uploaded to card as attachment)
│   └── Video Title 2.mp4 (also uploaded to card as attachment)
├── card_456_CardName2/
│   └── Another Video.mp4 (also uploaded to card as attachment)
└── results_summary_list123.json
```

**Note**: When `keep_local_files=False` (default), local video files are deleted after successful upload to keep storage clean.

## Complete Workflow

1. **Scan Trello List**: Fetches all cards from the specified source list
2. **Extract URLs**: Finds YouTube URLs in card titles and descriptions
3. **Download Videos**: Uses yt-dlp to download videos (optimized for <250MB to meet Trello limits)
4. **Upload to Cards**: Attaches downloaded videos directly to their source Trello cards
5. **Move Cards**: Optionally moves successfully processed cards to a destination list (e.g., "Completed")
6. **Cleanup**: Optionally deletes local files after successful upload
7. **Report**: Generates detailed summary of all operations including moves

## Important Notes

- **File Size Limit**: Trello has a 250MB attachment limit, so videos are downloaded in high quality while staying within this limit
- **Quality**: Videos are downloaded at up to 1080p, with preference for best quality under 250MB
- **Storage**: By default, local files are deleted after upload to save space (configurable)
- **Card Movement**: Successfully processed cards can be automatically moved to another list for workflow management
- **Error Handling**: Failed downloads, uploads, or card moves are tracked and reported separately

## YouTube URL Formats Supported

- `https://www.youtube.com/watch?v=VIDEO_ID`
- `https://youtu.be/VIDEO_ID`
- `https://www.youtube.com/embed/VIDEO_ID`
- `https://www.youtube.com/v/VIDEO_ID`
- `https://www.youtube.com/shorts/VIDEO_ID`
- All variations with/without `www` and `https`